In [2]:
import csv
import json
import base64
import io
import re
import logging
import sys
from pypdf import *
from bs4 import BeautifulSoup
from pdfreader import PDFDocument, SimplePDFViewer


from urllib.request import urlopen

key = "0b97b042aa1394ecbb0589b409bd7da2"

In [12]:
#Final methods to move to CRC
import csv
import string
import json
import base64
import io
import re
import logging
import sys
from pypdf import *
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')

def get_bill_ids(filePath):
    """Get bill ids from a csv file"""

    bill_ids = []
    with(open(filePath, 'r')) as f:
        reader = csv.reader(f)
        for row in reader:
            bill_ids.append(row[0])
    return bill_ids 

def get_text_from_bill_id(bill_id):
    """API pull to get text from bill id
    returns None if bill id is not found
    returns empy string if no text is unavailable
    returns encoded text (with base64) if available
    """
    
    urlGetBill = f"https://api.legiscan.com/?key={key}&op=getBill&id={bill_id}"
    r1 = urlopen(urlGetBill).read().decode()
    data1 = json.loads(r1)
    if data1["status"] == "ERROR":
        return None
    if data1["bill"]["texts"] == []:
        return ""
    else:
        doc_id = data1["bill"]["texts"][0]["doc_id"]
        urlGetText = f"https://api.legiscan.com/?key={key}&op=getBillText&id={doc_id}"
        r2 = urlopen(urlGetText).read().decode()
        data2 = json.loads(r2)
        textEncoded = data2["text"]["doc"]
        return textEncoded
    
def clean_text(text):
    """
    Clean the text by removing extra white spaces and sequences of numbers
    """
    pattern = r'\b(\d+\s+)+\d+\b'
    text = re.sub(pattern, '', text) 
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text) 
    # Regular expression to match sequences like '- 2 -'
    dash_number_pattern = r'-\s*\d+\s*-'
    text = re.sub(dash_number_pattern, '', text)
    return text  

def html_decode(data):
    # HTML text to plain text https://stackoverflow.com/questions/328356/extracting-text-from-html-file-using-python
    """
    Decode a base64 representation of a html type bill into string
    """
    decoded = io.BytesIO(base64.b64decode(data))
    soup = BeautifulSoup(decoded, 'html.parser')
    for script in soup(["script", "style"]):
        script.extract()

    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    text = clean_text(text)
    return text 

def pdf_decode(data):
    """
    Decode a base64 representation of a pdf type bill into string
    """
    decoded = base64.b64decode(data)
    pdfBuffer = io.BytesIO(decoded)
    reader = PdfReader(pdfBuffer)
    text = ""
    for i in range(len(reader.pages)):
        page = reader.pages[i]
        text += page.extract_text()

    text = clean_text(text)
    return text

def preprocess_text(decodedText):
    """
    Preprocess the text by removing punctuation, converting to lowercase, tokenizing, and removing stopwords
    """
    # Remove punctuation
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", decodedText)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word not in stop_words]
    # Join words back into a single string
    cleaned_text = " ".join(filtered_words)

    weird_pattern = r'\s*§+[\w\d]+\s*' # Remove weird characters
    cleaned_text = re.sub(weird_pattern, '', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) # Remove extra white spaces

    cleaned_text = clean_text(cleaned_text) #one last clean

    return cleaned_text

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\chess\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Sample Texts to see how it is formated:

MN and PA uses pdf_decode
VA and CA uses html_decode

Sample Texts:
CAhtml2TextPreprocessTest.txt
VAhtml2TextPreprocessTest.txt
MNpdf2TextPreprocessTest.txt
PApdf2TextPreprocessTest.txt


In [ ]:
VA_bill_ids = get_bill_ids("VA/2024-2024_Regular_Session/csv/bills.csv")
PA_bill_ids = get_bill_ids("PA/2023-2024_Regular_Session/csv/bills.csv")
MN_bill_ids = get_bill_ids("MN/2023-2024_93rd_Legislature/csv/bills.csv")
CA_bill_ids = get_bill_ids("CA/2023-2024_Regular_Session/csv/bills.csv")


In [ ]:
#MN Bill Preprocessing RUN THIS CODE
#Create a CSV file with the bill_id and the encoded text
with open("MN_bill_texts_encoded.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows([["bill_id", "text"]])
    bill_texts = []
    # print(bill_texts)
    count = 0
    for bill_id in MN_bill_ids: #skip the first line since its the header
        if bill_id == "bill_id":
            continue
        billText = get_text_from_bill_id(bill_id)
        # decodedText = pdf_decode(billText)
        # preprocessed_text = preprocess_text(decodedText)
        count += 1 #If empty string then text was unavailable on LegiScan
        # bill_texts.append([bill_id, billText])
        writer.writerows([[bill_id, billText]])
        # print(bill_id)
        if count % 100 == 0:
            print(count, bill_id)
        # if count % 100 == 0:
        #     print(count, bill_id)
# for bill_id in PA_bill_ids: #skip the first line since its the header
#     if bill_id == "bill_id":
#         continue
#     billText = get_text_from_bill_id(bill_id)
#     bill_texts.append(["PA", bill_id, billText]) #If empty string then text was unavailable on LegiScan
    


100 1643679
200 1646113
300 1654606
400 1661364


In [ ]:
#open CSV and decode, clean, and preprocess text from MN_bill_texts_encoded.csv

with open("MN_bill_texts_encoded.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        bill_id = row[0]
        text = row[1]
        if text == "":
            continue
        if text is None:
            continue
        else:
            text = pdf_decode(text)
        preprocessed_text = preprocess_text(text)
        with open("MN_bill_texts_preprocessed_100.csv", "w", newline='') as f:
            writer = csv.writer(f)
            writer.writerow([bill_id, preprocessed_text])

# with open("MN_bill_texts_encoded.csv", "r") as f:
#     reader = csv.reader(f)
#     next(reader)
#     for row in reader:
#         bill_id = row[0]
#         text = row[1]
#         if text == "":
#             continue
#         if text is None:
#             continue
#         else:
#             text = pdf_decode(text)
#         preprocessed_text = preprocess_text(text)
#         with open("MN_bill_texts_preprocessed.csv", "a") as f:
#             writer = csv.writer(f)
#             writer.writerow([bill_id, preprocessed_text])

In [ ]:
#PA Bill Preprocessing
#Create a CSV file with the bill_id and the encoded text
bill_texts = []
bill_texts.append(["bill_id", "text"])
count = 0
for bill_id in PA_bill_ids: #skip the first line since its the header
    if bill_id == "bill_id":
        continue
    billText = get_text_from_bill_id(bill_id)
    count += 1
    if count % 100 == 0:
        print(count)
    bill_texts.append([bill_id, billText]) #If empty string then text was unavailable on LegiScan

# for bill_id in PA_bill_ids: #skip the first line since its the header
#     if bill_id == "bill_id":
#         continue
#     billText = get_text_from_bill_id(bill_id)
#     bill_texts.append(["PA", bill_id, billText]) #If empty string then text was unavailable on LegiScan
    
with open("PA_bill_texts_encoded.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(bill_texts)


In [ ]:
#VA Bill Preprocessing
#Create a CSV file with the bill_id and the encoded text
bill_texts = []
bill_texts.append(["bill_id", "text"])
count = 0
for bill_id in VA_bill_ids: #skip the first line since its the header
    if bill_id == "bill_id":
        continue
    billText = get_text_from_bill_id(bill_id)
    count += 1
    if count % 100 == 0:
        print(count)
    bill_texts.append([bill_id, billText]) #If empty string then text was unavailable on LegiScan

# for bill_id in PA_bill_ids: #skip the first line since its the header
#     if bill_id == "bill_id":
#         continue
#     billText = get_text_from_bill_id(bill_id)
#     bill_texts.append(["PA", bill_id, billText]) #If empty string then text was unavailable on LegiScan
    
with open("VA_bill_texts_encoded.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(bill_texts)